In [1]:
import random
import torch
from d2l import torch as d2l

生成数据集

通过一个带有噪声的线性模型来生成数据
其中w代表权重向量，b代表偏置项，num_examples代表要生成的样本数量

torch.normal() 函数用于生成服从正态分布（高斯分布）的随机数。
y.reshape((-1, 1))   -1是一个特殊值，意味着让pytorch自动计算该维度的大小以保持原数总数不变

调用下面编写的函数会返回一个特征矩阵和一个标签向量

In [2]:
def synthetic_data(w, b, num_examples):  #@save
    """生成y=Xw+b+噪声"""
    X = torch.normal(0, 1, (num_examples, len(w)))  # 通过正态分布，生成特征矩阵
    y = torch.matmul(X, w) + b  # 通过matmul矩阵乘法，生成标签
    y += torch.normal(0, 0.01, y.shape)  # 添加噪声
    return X, y.reshape((-1, 1))  # 通过reshape来确保输出标签是列向量形式

true_w = torch.tensor([2, -3.4])
true_b = 4.2
features, labels = synthetic_data(true_w, true_b, 1000)
#因为传入的w是二维的，也就是一个样本点的大小是二维的(有两个特征)

读取数据集

该函数接收批量大小、特征矩阵和标签向量作为输入，生成大小为batch_size的小批量。 每个小批量包含一组特征和标签。

特征矩阵，形状为(样本数量，特征维度)
标签向量，形状为(样本数量，1)

其中features[batch_indices] 是按照索引列表进行取值的操作，这在 PyTorch 中被称为索引选择或高级索引
features[batch_indices] 会返回一个新的张量，其中包含了 features 中对应这些索引位置的行

In [3]:
def data_iter(batch_size, features, labels):
    num_examples = len(features)
    indices = list(range(num_examples))  # 创建索引列表
    # 这些样本是随机读取的，没有特定的顺序
    random.shuffle(indices)  # 随机打乱样本顺序
    for i in range(0, num_examples, batch_size):
        batch_indices = torch.tensor(
            indices[i: min(i + batch_size, num_examples)])
        yield features[batch_indices], labels[batch_indices]  # 惰性计算，只在需要时生成批次数据

初始化模型参数，定义损失函数

In [ ]:
w = torch.normal(0, 0.01, size=(2,1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

def squared_loss(y_hat, y):  #@save
    """均方损失"""
    return (y_hat - y.reshape(y_hat.shape)) ** 2 / 2

定义优化算法

parms 代表需要优化的参数列表
lr 代表学习率
batch_size 代表小批量的大小，用于归一化

with torch.no_grad() 是一个上下文管理器(context manager)，在这个上下文中执行的操作不会被记录用于反向传播
具体来说，会禁用梯度计算（临时关闭自动求导机制，不会小号额外的内存来存储计算历史）

In [5]:
def sgd(params, lr, batch_size):  #@save
    """小批量随机梯度下降"""
    with torch.no_grad():
        for param in params:
            param -= lr * param.grad / batch_size
            param.grad.zero_()

训练

In [7]:
def linreg(X, w, b):  #@save
    """定义线性回归模型"""
    return torch.matmul(X, w) + b

def squared_loss(y_hat, y):  #@save
    """均方损失"""
    return (y_hat - y.reshape(y_hat.shape)) ** 2 / 2

# 超参数设置
batch_size = 10

# 读取数据集
# for X, y in data_iter(batch_size, features, labels):
#     print(X, '\n', y)
#     break

lr = 0.03
num_epochs = 3  # 训练轮数
net = linreg  # 模型函数
loss = squared_loss  # 损失函数，这里使用均方误差

for epoch in range(num_epochs):
    for X, y in data_iter(batch_size, features, labels):  # 读取小批量数据
        l = loss(net(X, w, b), y)  # X和y的小批量损失，前向传播
        # 因为l形状是(batch_size,1)，而不是一个标量。将l中的所有元素加到一起，并以此计算关于[w,b]的梯度
        l.sum().backward() # 反向传播，计算梯度
        sgd([w, b], lr, batch_size)  # 使用参数的梯度更新参数
    with torch.no_grad():  # 评估当前训练效果，此时不需要记录操作
        train_l = loss(net(features, w, b), labels)
        print(f'epoch {epoch + 1}, loss {float(train_l.mean()):f}')

epoch 1, loss 0.034151
epoch 2, loss 0.000120
epoch 3, loss 0.000046


因为是自行生成的数据，因此可以直接进行比较

In [8]:
print(f'w的估计误差: {true_w - w.reshape(true_w.shape)}')
print(f'b的估计误差: {true_b - b}')

w的估计误差: tensor([ 0.0007, -0.0003], grad_fn=<SubBackward0>)
b的估计误差: tensor([-8.3447e-05], grad_fn=<RsubBackward1>)
